<a href="https://colab.research.google.com/github/walidbrini/TensorFlow/blob/main/TFtuto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/GoogleCloudPlatform/practical-ml-vision-book.git

Cloning into 'practical-ml-vision-book'...
remote: Enumerating objects: 1347, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (293/293), done.
remote: Total 1347 (delta 29), reused 309 (delta 22), pack-reused 1031
Receiving objects: 100% (1347/1347), 289.29 MiB | 28.50 MiB/s, done.
Resolving deltas: 100% (620/620), done.
Checking out files: 100% (400/400), done.


In [5]:
import tensorflow as tf
import matplotlib as plt 

In [6]:
#Read and decode image 

def read_and_decode(filename, reshape_dims):
 # 1. Read the file.
 img = tf.io.read_file(filename)
 # 2. Convert the compressed string to a 3D uint8 tensor.
 img = tf.image.decode_jpeg(img, channels=3)
 # 3. Convert 3D uint8 to floats in the [0,1] range.
 img = tf.image.convert_image_dtype(img, tf.float32)
 # 4. Resize the image to the desired size.
 return tf.image.resize(img, reshape_dims)

In [7]:
def show_image(filename):
 img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
 plt.imshow(img.numpy());


In [ ]:
print(type(read_and_decode(gs://cloud-ml-data/)))